In [47]:
import json

In [48]:
data = json.loads(open('new_desc.json', encoding='utf-8').read())
data = [json.loads(x) for x in data]
endata = data[::2]
chdata = data[1::2]
assert len(endata) == len(chdata)

In [49]:
allres = {}
ename_to_cname = {}
version = 'X.X'
for ed, cd in zip(endata, chdata):
    for [ek, ev], [ck, cv] in zip(ed, cd):
        k = ek
        if k[:5] == 'SKILL':
            ori_k = k
            name = k.split('_')[1]
            allres['CHARACTER/' + name] = {}
            k = k.replace('NORMAL ATTACK', 'NORMAL_ATTACK')
            k = k.replace('ELEMENTAL SKILL', 'ELEMENTAL_SKILL')
            k = k.replace('ELEMENTAL BURST', 'ELEMENTAL_BURST')
            k = k.replace('PASSIVE SKILL', 'PASSIVE')
            mat = {
                'NORMAL_ATTACK': '普通攻击',
                'ELEMENTAL_SKILL': '元素战技',
                'ELEMENTAL_BURST': '元素爆发',
                'PASSIVE': '被动技能',
            }
            c_skill_type = None
            for etype in mat:
                if etype in k:
                    c_skill_type = mat[etype]
                    break
            ename_to_cname[k] = ck
            allres[k] = ev
            allres[k]['names'].update(cv['names'])
            version = list(allres[k]['descs'].keys())[0]
            allres[k]['descs'][version].update(cv['descs'][version])
        else:
            ename_to_cname[k] = ck
            allres[k] = ev
            allres[k]['names'].update(cv['names'])
            version = list(allres[k]['descs'].keys())[0]
            allres[k]['descs'][version].update(cv['descs'][version])
# print(json.dumps(allres, indent = 2, ensure_ascii = False))

In [50]:
new_card_data = json.load(open('new_cards.json'))

In [51]:
for image_path, file_name, name, id, URL in new_card_data:
    id = int(id)
    keys = list(allres.keys())
    selected_keys = [x for x in keys if name in x and 'CHARACTER/' not in x]
    if id < 100000:
        # is character
        cname = ename_to_cname[selected_keys[0]].split('_')[1]
        allres['CHARACTER/' + name] = {
          'names': {
              'en-US': name,
              'zh-CN': cname,
          },
          'descs': {
              version: {
                  'en-US': '',
                  'zh-CN': ''
              }
          },
          'image_path': image_path,
          'id': id,
        }
        continue
    if len(selected_keys) != 1:
        print('Cannot find', name, 'in', selected_keys)
        continue
    key = selected_keys[0]
    allres[key]['image_path'] = image_path
    allres[key]['id'] = id

["SKILL_Kuki Shinobu_NORMAL_ATTACK/Shinobu's Shadowsword", 'SKILL_Kuki Shinobu_ELEMENTAL_SKILL/Sanctifying Ring', 'SKILL_Kuki Shinobu_ELEMENTAL_BURST/Gyoei Narukami Kariyama Rite', 'TALENT_Kuki Shinobu/To Ward Weakness']
['SKILL_Faruzan_NORMAL_ATTACK/Parthian Shot', 'SKILL_Faruzan_ELEMENTAL_SKILL/Wind Realm of Nasamjnin', "SKILL_Faruzan_ELEMENTAL_BURST/The Wind's Secret Ways", 'TALENT_Faruzan/The Wondrous Path of Truth']
['SKILL_Abyss Herald: Wicked Torrents_NORMAL_ATTACK/Rippling Slash', 'SKILL_Abyss Herald: Wicked Torrents_ELEMENTAL_SKILL/Vortex Edge', 'SKILL_Abyss Herald: Wicked Torrents_ELEMENTAL_BURST/Torrential Shock', 'SKILL_Abyss Herald: Wicked Torrents_PASSIVE/Watery Rebirth', 'SKILL_Abyss Herald: Wicked Torrents_ELEMENTAL_SKILL/Rippling Blades']
['SKILL_Emperor of Fire and Iron_NORMAL_ATTACK/Shatterclamp Strike', 'SKILL_Emperor of Fire and Iron_ELEMENTAL_SKILL/Buster Blaze', 'SKILL_Emperor of Fire and Iron_ELEMENTAL_BURST/Battle-Line Detonation', 'SKILL_Emperor of Fire and Ir

In [52]:
json.dump(allres, open('new_result.json', 'w', encoding='utf-8'), 
          ensure_ascii=False, indent=4)